### Make the stuff do the thing

In [1]:
import json
import os
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
# import model, sample, encoder
!pip install gpt-2-simple 
import gpt_2_simple as gpt2

model_name = "345M" # The GPT-2 model we're using

gpt2.download_gpt2(model_name=model_name) # Download the model

     |████████████████████████████████| 645kB 23.9MB/s 
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7-cp36-none-any.whl size=23557 sha256=8054885574e76102e3604e91c9cdbac9ae6cc48c7a104c8c6eb94edf5d3bc335
  Stored in directory: /root/.cache/pip/wheels/28/7f/89/1253cc7ae7fd1cdf130fa146ab17314fd2a5a6d48ccf21dec5
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 159Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 89.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 252Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:18, 74.9Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 427Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 94.6Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 145Mit/s]                                                       


### Finetune Model
Training the model on a custom dataset (50 positive and 50 negative samples from the stanford sentiment analysis dataset)

In [2]:
file_name = "sentiment50.txt" # where the dataset is located

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              steps=400)   # steps is max number of training steps


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/345M/model.ckpt
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  3.15it/s]


dataset has 29942 tokens
Training...
[1 | 77.48] loss=3.39 avg=3.39
[2 | 141.33] loss=3.50 avg=3.44
[3 | 205.49] loss=3.03 avg=3.31
[4 | 269.56] loss=3.51 avg=3.36
[5 | 333.40] loss=3.38 avg=3.36
[6 | 397.17] loss=3.40 avg=3.37
[7 | 461.15] loss=2.65 avg=3.26
[8 | 525.45] loss=3.65 avg=3.31
[9 | 590.01] loss=2.86 avg=3.26
[10 | 654.03] loss=3.41 avg=3.28
[11 | 717.95] loss=3.17 avg=3.27
[12 | 781.84] loss=3.06 avg=3.25
[13 | 845.87] loss=2.16 avg=3.16
[14 | 909.98] loss=2.51 avg=3.11
[15 | 974.17] loss=4.21 avg=3.19
[16 | 1038.08] loss=2.84 avg=3.16
[17 | 1102.12] loss=3.02 avg=3.15
[18 | 1166.25] loss=2.46 avg=3.11
[19 | 1230.57] loss=2.64 avg=3.09
[20 | 1294.70] loss=2.26 avg=3.04
[21 | 1358.72] loss=3.52 avg=3.07
[22 | 1422.63] loss=3.58 avg=3.09
[23 | 1486.45] loss=2.26 avg=3.05
[24 | 1550.27] loss=3.34 avg=3.06
[25 | 1614.11] loss=2.91 avg=3.06
[26 | 1677.34] loss=3.09 avg=3.06
[27 | 1739.31] loss=3.17 avg=3.06
[28 | 1803.11] loss=3.00 avg=3.06
[29 | 1867.54] loss=2.75 avg=3.05
[3

### Cool Stuff

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

### Use It
In the prompt, prefix it with // and suffix it with || to get the model to respond with positive or negative


In [0]:
!git clone https://github.com/openai/gpt-2.git
import os
os.chdir("gpt-2/src/")
import tensorflow as tf
import model, sample, encoder
os.chdir('../../..')


In [0]:
!ls
interact_model(
    'run1',
    None,
    1,
    1,
    2,
    1,
    0,
    './content/checkpoint'
)

In [0]:
import shutil
shutil.make_archive("gpt2-sent50-400", 'zip', "/content/checkpoint")